In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import tensorflow as tf
import torch.utils.data
from torch.utils.data import TensorDataset
from transformers import TFRobertaModel, RobertaTokenizer


In [ ]:
from google.colab import files
upload= files.upload()

Saving finalset.xlsx to finalset (2).xlsx


In [ ]:
df=pd.read_excel("finalset.xlsx")
category_map={
    "Knowledge": 0,
    "Comprehension": 1,
    "Application": 2,
    "Analysis": 3,
    "Synthesis": 4,
    "Evaluation": 5
}
df["category"]=df["category"].map(category_map)
df.to_excel("final.xlsx",index=False)

In [ ]:
df.head()

,category,text
0,0,List two reference parameters in the setHour ...
1,0,Explain briefly the meaning of the following ...
2,0,Label the parts of the diagram
3,0,"4. Based on the above dataType class, list all..."
4,0,5. Define morphology


In [ ]:
df['category'].value_counts()

4    130
0    126
5    124
1    123
3    123
2    115
Name: category, dtype: int64

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')


In [ ]:
token= tokenizer.encode_plus(
    df['text'].iloc[0],
    max_length=256,
    truncation=True,
    padding='max_length',
    add_special_tokens=True,
    return_tensors='tf'
)
token.pop('token_type_ids', None) 

In [ ]:
X_input_ids= np.zeros((len(df),256))
X_attn_masks= np.zeros((len(df),256))

In [ ]:
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['text'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256,
            truncation=True,
            padding='max_length',
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text['input_ids']
        masks[i, :] = tokenized_text['attention_mask']
    return ids, masks
    

In [ ]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)



0it [00:00, ?it/s]

In [ ]:
labels= np.zeros((len(df),6))
labels.shape

(741, 6)

In [ ]:
labels[np.arange(len(df)), df['category'].values] = 1


In [ ]:
labels

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]])

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(6,), dtype=tf.float64, name=None))>

In [ ]:
def BloomsDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [ ]:
dataset = dataset.map(BloomsDatasetMapFunction)

In [ ]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(6,), dtype=tf.float64, name=None))>

In [ ]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True)

In [ ]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 6), dtype=tf.float64, name=None))>

In [ ]:
p=0.8
train_size=int((len(df)//16*p))

In [ ]:
train_size

36

In [ ]:
train_dataset=dataset.take(train_size)
val_dataset=dataset.skip(train_size)
len(train_dataset),len(val_dataset)


(36, 10)

In [ ]:
from transformers import RobertaTokenizer, TFRobertaModel

roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

roberta_embds = roberta_model(input_ids, attention_mask=attn_masks)[1] 
intermediate_layer = tf.keras.layers.Dense(256, activation='relu', name='roberta_intermediate_layer')(roberta_embds)
output_layer = tf.keras.layers.Dense(6, activation='softmax', name='roberta_output_layer')(intermediate_layer) 

Blooms_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
Blooms_model.summary()


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 el)                            thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [ ]:
optim = tf.keras.optimizers.legacy.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [ ]:
Blooms_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [ ]:
hist = Blooms_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=5
)

Epoch 1/5
36/36 [==============================] - 2091s 58s/step - loss: 1.8010 - accuracy: 0.1753 - val_loss: 1.7546 - val_accuracy: 0.3313
Epoch 2/5
36/36 [==============================] - 2120s 59s/step - loss: 1.5717 - accuracy: 0.4427 - val_loss: 1.0178 - val_accuracy: 0.7500
Epoch 3/5
36/36 [==============================] - 2123s 59s/step - loss: 0.9113 - accuracy: 0.7014 - val_loss: 0.4745 - val_accuracy: 0.8562
Epoch 4/5
36/36 [==============================] - 2190s 61s/step - loss: 0.5563 - accuracy: 0.8281 - val_loss: 0.2709 - val_accuracy: 0.9250
Epoch 5/5
36/36 [==============================] - 2095s 58s/step - loss: 0.3550 - accuracy: 0.8958 - val_loss: 0.2323 - val_accuracy: 0.9187
